# DATA LOAD

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import warnings
import seaborn as sns

warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# travel_22 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/data/2022년_국민여행조사_원자료_국내여행/DATA_2022년 국민여행조사.xlsx')

In [4]:
# travel_22.head(5)

In [11]:
travel_22 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/data/2022년_국민여행조사_원자료_국내여행/관광휴양_혼행객_22.xlsx')

In [ ]:
# cols = ['D_TRA1_CASE', 'D_TRA2_CASE', 'D_TRA3_CASE', 'D_TRA4_CASE', 'D_TRA5_CASE']

# # 값이 한번이라도 2인 행만 필터링: 관광/휴향 목적 여행인 사람 필터링
# travel_heal = travel_22[travel_22[cols].eq(1).any(axis=1)]

# col = ['D_TRA1_NUM', 'D_TRA2_NUM', 'D_TRA3_NUM', 'D_TRA4_NUM', 'D_TRA5_NUM']

# # # 값이 한번이라도 1인 행만 필터링: 혼자 여행을 해 본 사람 필터링
# travel_heal_1 = travel_heal[travel_heal[col].eq(1).any(axis=1)]

# print(len(travel_heal_1))

In [ ]:
# travel_heal_1.head(5)

In [ ]:
# travel_heal_1.info()

In [ ]:
# travel_heal_1.to_csv('22년_1인_관광_.csv', index=False)
# from google.colab import files
# # files.download('22년_1인_관광_.csv')

## 혼행객의 여행 특성 파악

### 혼자 여행한 case에 대해서 wide format을 long format으로 변경

In [12]:
phases = [1, 2, 3]
days = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]

dfs = []

for i in phases:
    for j in days:
        col_name = f'D_TRA{i}_{j}_SPOT'
        if col_name in travel_22.columns:
            df_temp = travel_22[['ID']].copy()
            df_temp['차수'] = i
            df_temp['여행 인원'] = travel_22.get(f'D_TRA{i}_NUM')
            df_temp['1인 지출 비용'] = travel_22.get(f'D_TRA{i}_ONE_COST')
            df_temp['방문 지역'] = travel_22.get(col_name)
            df_temp['여행 일수'] = travel_22.get(f'D_TRA{i}_S_Day')
            dfs.append(df_temp)

long_df = pd.concat(dfs, ignore_index=True)
long_df = long_df.dropna(subset=['방문 지역'])
alone_df = long_df[(long_df['여행 인원'] == 1)]


print(alone_df.head())
print(f"혼자 여행한 케이스 수: {len(alone_df)}")

                   ID  차수 여행 인원 1인 지출 비용  방문 지역 여행 일수
0  11010630151_328440   1     1    50000  37360     1
1  1104055115A_302779   1     1   515000  37010     1
2  1104067041A_404480   1     1   400000  36020     2
3  11050530101_429702   1     1   200000  31370     0
4  1105063193A_556053   1     1   456000  33350     1
혼자 여행한 케이스 수: 6341


### 혼행객의 평균 지출

In [13]:
import plotly.express as px

fig = px.histogram(
    alone_df,
    x='1인 지출 비용',
    nbins=40,
    title='혼행객의 1인 지출 비용 분포',
    labels={'1인 지출 비용': '1인 지출 비용 (원)'}
)

mean_cost = alone_df['1인 지출 비용'].mean()
fig.add_vline(
    x=mean_cost,
    line_dash='dash',
    line_color='red',
    annotation_text=f'평균: {mean_cost:,.0f} 원',
    annotation_position='top right'
)

fig.show()

### 혼행객의 주요 방문지역

In [14]:
city_df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/data/2022년_국민여행조사_원자료_국내여행/code.xlsx')
#city_df.head()

city_df['시/도 코드'] = city_df['시/도 코드'].astype(str).str.zfill(2)
city_df['시/군/구 코드'] = city_df['시/군/구 코드'].astype(str).str.zfill(3)

alone_df['방문 지역'] = alone_df['방문 지역'].astype(str).str.zfill(5)
alone_df['시도 코드'] = alone_df['방문 지역'].str[:2]
alone_df['시군구 코드'] = alone_df['방문 지역'].str[2:]

alone_df = alone_df.merge(
    city_df,
    left_on=['시도 코드', '시군구 코드'],
    right_on=['시/도 코드', '시/군/구 코드'],
    how='left'
)

alone_df['지역명'] = alone_df['시/도명'] + ' ' + alone_df['시/군/구명']

print(alone_df[['방문 지역', '지역명']].head())

   방문 지역       지역명
0  37360  경상북도 청도군
1  37010  경상북도 포항시
2  36020  전라남도 여수시
3  31370   경기도 가평군
4  33350  충청북도 진천군


In [15]:
# 코드 숫자 앞자리 0 보존을 위해 문자열 처리
city_df['시/도 코드'] = city_df['시/도 코드'].astype(str).str.zfill(2)
city_df['시/군/구 코드'] = city_df['시/군/구 코드'].astype(str).str.zfill(3)

# 동코드, 시도군명 생성
city_df['동코드'] = city_df['시/도 코드'] + city_df['시/군/구 코드']
city_df['시도군명'] = city_df['시/도명'] + ' ' + city_df['시/군/구명']

# 필요한 열만 선택해서 새 파일로 저장
new_df = city_df[['동코드', '시도군명']]
new_df.to_excel('new_code.xlsx', index=False)

In [ ]:
# from google.colab import files
# files.download('new_code.xlsx')

In [16]:
# 방문 지역별 빈도수
region_counts = alone_df['지역명'].value_counts().reset_index()
region_counts.columns = ['지역명', '방문자 수']

fig = px.bar(
    region_counts,
    x='지역명',
    y='방문자 수',
    title='1인 여행객의 방문 지역 빈도수',
    labels={'지역명': '지역명', '방문자 수': '방문 횟수'},
    color='방문자 수',
    color_continuous_scale='Blues'
)

fig.update_layout(xaxis_tickangle=-45)
fig.show()

In [17]:
# 상위 10개 지역에 대한 방문 비율
top10 = region_counts.head(10)

fig = px.pie(
    top10,
    names='지역명',
    values='방문자 수',
    title='1인 여행객 상위 10개 방문 지역 비율'
)
fig.show()

In [18]:
# 하위 10개 지역에 대한 방문 비율
bottom10 = region_counts.tail(10)\

fig = px.pie(
    bottom10,
    names='지역명',
    values='방문자 수',
    title='1인 여행객 방문이 가장 적은 10개 지역 비율'
)
fig.show()

In [19]:
# 시/도명 기준 방문 횟수 비율
city_ratio = alone_df['시/도명'].value_counts(normalize=True).reset_index()
city_ratio.columns = ['시/도명', '방문 비율']
city_ratio['방문 비율'] *= 100

fig = px.bar(
    city_ratio,
    x='시/도명',
    y='방문 비율',
    title='혼자 여행 시 시/도별 방문 비율',
    labels={'방문 비율': '방문 비율 (%)'},
    text='방문 비율'
)

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(
    xaxis_tickangle=-45,
    yaxis_range=[0, city_ratio['방문 비율'].max() + 5],
    height=500
)
fig.show()

### 혼행객의 평균 여행 일수

In [20]:
fig = px.histogram(
    alone_df,
    x='여행 일수',
    nbins=15,
    title='혼자 여행 시 여행 일수 분포',
    labels={'여행 일수': '여행 일수 (일)'}
)

fig.add_vline(
    x = alone_df['여행 일수'].mean(),
    line_dash = 'dash',
    line_color = 'red',
    annotation_text = f"평균: {alone_df['여행 일수'].mean():.1f}일",
    annotation_position = "top right"
)

fig.show()

## 혼행객의 교통 접근성과 지역소비 간의 상관관계 분석

### 혼행객의 대중교통 이용 비율 확인

In [21]:
import plotly.express as px

travel_alone = travel_22[travel_22['A7'] == 1]

all_transport = pd.concat([
    travel_alone['A1_1'],
    travel_alone['A1_2'],
    travel_alone['A1_3']
])

all_transport = all_transport.replace(' ', pd.NA)
all_transport = all_transport.dropna().astype(int)

# 혼행객의 주요 이동수단 선택 비율
transport_counts = all_transport.value_counts().sort_index()
df_transport = transport_counts.reset_index()
df_transport.columns = ['Transport', 'Count']

transport_map = {
    1: '자가용', 2: '철도', 3: '힝공기', 4: '선박/해상 교통',
    5: '지하철', 6: '고속/시외/시내버스', 7: '전세/관광버스', 8: '차량대여/렌트',
    9: '자전거', 10: '택시', 11: '기타'
}
df_transport['Transport'] = df_transport['Transport'].map(transport_map)

fig = px.pie(df_transport, names='Transport', values='Count',
             title='혼행객의 주요 이동수단 선택 비율',
             color_discrete_sequence=px.colors.qualitative.Set3,
             hole = 0.0)

fig.update_traces(textposition='outside', textinfo='percent+label')
fig.show()

In [22]:
# 혼행객의 개인 교통수단과 대중교통 이용 비율
def categorize_transport(x):
    if x in [1, 8, 10]:
        return '개인 교통수단'
    else:
        return '대중교통'

transport_category = all_transport.map(categorize_transport)

category_counts = transport_category.value_counts().reset_index()
category_counts.columns = ['TransportType', 'Count']

fig = px.pie(category_counts, names='TransportType', values='Count',
             title='혼행객 교통수단 유형 비율',
             color_discrete_sequence=px.colors.qualitative.Set2,
             hole=0.0)

fig.update_traces(textposition='outside', textinfo='percent+label')
fig.show()

### 대중교통 이용자의 교통 만족도 분포

In [23]:
public_users = travel_alone[
    travel_alone['A1_1'].astype(str).isin(['2','3','4','5', '6', '7', '9', '11']) |
    travel_alone['A1_2'].astype(str).isin(['2','3','4','5', '6', '7', '9', '11']) |
    travel_alone['A1_3'].astype(str).isin(['2','3','4','5', '6', '7', '9', '11'])
]

satisfaction = public_users['A10A_3'].replace(' ', pd.NA).dropna().astype(int)
satis_counts = satisfaction.value_counts().sort_index()

label_map = {2: '불만족', 3: '보통', 4: '만족', 5: '매우만족'}
df_satis = satis_counts.rename(index=label_map).reset_index()
df_satis.columns = ['만족도', '응답자수']

# 혼행객 중 대중교통 이용자의 교통만족도 분포
fig = px.bar(df_satis, x='만족도', y='응답자수',
             color='만족도', text='응답자수',
             title='대중교통 이용자의 교통 만족도 분포',
             color_discrete_sequence=px.colors.qualitative.Set2)

fig.update_layout(showlegend=False)
fig.update_traces(textposition='outside')
fig.show()

In [24]:
# 혼행객 중 대중교통 이용자의 교통만족도 비중
fig = px.pie(df_satis,
             names='만족도',
             values='응답자수',
             title='대중교통 이용자의 교통 만족도 분포',
             color_discrete_sequence=px.colors.qualitative.Pastel)

fig.update_traces(textposition='outside', textinfo='percent+label')
fig.show()

#### 혼행객의 개인교통수단과 대중교통 이용자간 여행지 지출비용 비교

- 결측치 처리? -> 0으로 변환 or 제거

In [25]:
missing_A9 = travel_alone['A9'].isna().sum()
print(f"총지출 결측치 개수: {missing_A9}")

missing_NA9 = travel_alone['NA9'].isna().sum()
print(f"1인 총지출 결측치 개수: {missing_NA9}")

총지출 결측치 개수: 0
1인 총지출 결측치 개수: 0


In [26]:
# 총 경비와 1인 경비가 동일한지를 확인
same = travel_alone[travel_alone['A9'] == travel_alone['NA9']]
print(f"경비가 동일한 사람 수: {len(same)}명")

not_same = travel_alone[travel_alone['A9'] != travel_alone['NA9']]
print(f"혼행객이나 경비가 다른 사람 수: {len(not_same)}명")

경비가 동일한 사람 수: 320명
혼행객이나 경비가 다른 사람 수: 53명


In [27]:
# 1순위 교통수단 기준으로 분류한 후 그룹별 1인당 평균 지출 계산
same['TransportType'] = same['A1_1'].map(lambda x: categorize_transport(int(x)) if pd.notna(x) else 'Unknown')
mean_spending = same.groupby('TransportType')['NA9'].mean()

print("혼자 여행자 그룹별 1인당 평균 지출비용:")
print(mean_spending)

혼자 여행자 그룹별 1인당 평균 지출비용:
TransportType
개인 교통수단    150279.781421
대중교통       141028.474453
Name: NA9, dtype: float64


In [28]:
df_mean = mean_spending.reset_index()
df_mean.columns = ['TransportType', '평균 1인당 지출비용']
df_mean['text'] = df_mean['평균 1인당 지출비용'].apply(lambda x: f"{round(x):,} 원")

fig = px.bar(
    df_mean,
    x='TransportType',
    y='평균 1인당 지출비용',
    color='TransportType',
    text='평균 1인당 지출비용',
    title='혼행객의 교통수단별 평균 1인당 지출비용',
    color_discrete_sequence=px.colors.qualitative.Set2
)

fig.update_traces(texttemplate='%{text}', textposition='outside')
fig.update_layout(showlegend=False)
fig.show()

### 혼행객의 교통비 비중

In [29]:
# 총 교통비와 1인 교통비가 동일한지를 확인
same_count = (travel_alone['NA9F'] == travel_alone['A9F']).sum()
diff_count = (travel_alone['NA9F'] != travel_alone['A9F']).sum()

print(f"같은 경우: {same_count}건")
print(f"다른 경우: {diff_count}건")

같은 경우: 358건
다른 경우: 15건


In [30]:
# 1인의 총 교통비 지출 분포 확인
fig = px.histogram(
    travel_alone,
    x='NA9F',
    nbins=30,
    title='1인 교통비 지출 분포',
    labels={'NA9F': '1인 교통비 지출 (원)'}
)
fig.update_layout(yaxis_title='빈도수')

mean_cost = travel_alone['NA9F'].mean()
fig.add_vline(
    x=mean_cost,
    line_dash='dash',
    line_color='red',
    annotation_text=f'평균: {mean_cost:,.0f} 원',
    annotation_position='top right'
)

fig.show()

In [31]:
# 교통 수단별 1인 지출 비중
transport_cols = ['NA9F_1', 'NA9F_2', 'NA9F_3', 'NA9F_4', 'NA9F_5']

filtered = travel_alone[travel_alone[transport_cols].sum(axis=1) > 0].copy()

for col in transport_cols:
    filtered[f'{col}_ratio'] = filtered[col] / filtered[transport_cols].sum(axis=1)

mean_ratios = filtered[[f'{col}_ratio' for col in transport_cols]].mean().reset_index()
mean_ratios.columns = ['교통수단', '평균 비중']
mean_ratios['교통수단'] = mean_ratios['교통수단'].str.replace('_ratio', '')
mean_ratios['교통수단명'] = mean_ratios['교통수단'].map({
    'NA9F_1': '철도',
    'NA9F_2': '항공',
    'NA9F_3': '수상',
    'NA9F_4': '도로',
    'NA9F_5': '자가용/렌트카'
})

fig = px.pie(
    mean_ratios,
    names='교통수단명',
    values='평균 비중',
    title='1인 교통비: 교통수단별 평균 비중',
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig.update_traces(textinfo='percent+label')
fig.show()

### 재방문의사 및 타인추천 의도와 교통비 지출 간의 상관관계

In [32]:
# 재방문 의도 vs 1인 교통비
fig1 = px.box(
    travel_alone,
    x='A11',
    y='NA9F',
    title='재방문 의도에 따른 1인 교통비 분포',
    labels={'A11': '재방문 의도', 'NA9F': '1인 교통비 (원)'}
)
fig1.update_layout(xaxis=dict(tickmode='linear'))
fig1.show()

# 타인추천 의도 vs 1인 교통비
fig2 = px.box(
    travel_alone,
    x='A12',
    y='NA9F',
    title='타인추천 의도에 따른 1인 교통비 분포',
    labels={'A12': '타인추천 의도', 'NA9F': '1인 교통비 (원)'}
)
fig2.update_layout(xaxis=dict(tickmode='linear'))
fig2.show()

In [33]:
mean_by_a11 = travel_alone.groupby('A11')['NA9F'].mean().reset_index()
mean_by_a12 = travel_alone.groupby('A12')['NA9F'].mean().reset_index()

fig3 = px.bar(
    mean_by_a11,
    x='A11',
    y='NA9F',
    title='재방문 의도별 평균 교통비',
    labels={'A11': '재방문 의도', 'NA9F': '평균 교통비 (원)'}
)
fig3.update_layout(xaxis=dict(tickmode='linear'))
fig3.show()

fig4 = px.bar(
    mean_by_a12,
    x='A12',
    y='NA9F',
    title='타인추천 의도별 평균 교통비',
    labels={'A12': '타인추천 의도', 'NA9F': '평균 교통비 (원)'}
)
fig4.update_layout(xaxis=dict(tickmode='linear'))
fig4.show()

### 교통비와 재방문의도/타인추천 의도의 스피어만 상관계수 분석

In [34]:
from scipy.stats import spearmanr

df_corr = travel_alone[['NA9F', 'A11', 'A12']].dropna()

rho_a11, pval_a11 = spearmanr(df_corr['NA9F'], df_corr['A11'])
rho_a12, pval_a12 = spearmanr(df_corr['NA9F'], df_corr['A12'])

print(f"[재방문 의도 vs 교통비] Spearman 계수: {rho_a11:.3f}, p-value: {pval_a11:.3f}")
print(f"[타인추천 의도 vs 교통비] Spearman 계수: {rho_a12:.3f}, p-value: {pval_a12:.3f}")


[재방문 의도 vs 교통비] Spearman 계수: 0.031, p-value: 0.550
[타인추천 의도 vs 교통비] Spearman 계수: 0.187, p-value: 0.000


- 재방문의도와 교통비의 Spearman 계수: 0.035임에 따라 상관이 거의 없으며 p-value도 0.502로 높은 값을 가지기 때문에 통계적으로 유의하지 않은 결과라 볼 수 있음
  - 즉 교통비와 재방문 의도는 연관성이 없음
- 타인추천의도와 교통의 Spearman 계수: 0.198로 약한 양의 상관관계를 띤다고 볼 수 있음(거의 없다고 볼수 있긴함) 이때 p-value는 0으로 굉장히 작기때문에 통계적으로 유의하다고 볼 수 있음
  - 즉 교통비가 많을 수록 타인에게 추천하려는 경향이 약간은 있으나 교통비가 너무 많으면 오히려 추천도가 떨어지는 것을 확인할 수 있음

## 혼행객 대상 관광패스 개발

### 여행사 상품 구매 여부

In [35]:
purchase_counts = travel_alone['A2'].value_counts().rename(index={1: '구매함', 2: '구매 안함'})

fig = px.pie(
    names=purchase_counts.index,
    values=purchase_counts.values,
    title='혼행객의 여행사 상품 구매 여부'
)
fig.update_traces(textinfo='percent+label')
fig.show()